Derived from https://arxiv.org/pdf/1711.07128.pdf


In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0119 11:56:32.226847  2180 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0119 11:56:32.230834  2180 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='ds_cnn'
start_checkpoint=None
logging_interval=10
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='1000,1000,1000' #Declare  the training steps for which the learning rates will be used
batch_size=256
model_size_info=[5, 64, 10, 4, 2, 2, 64, 3, 3, 1, 1, 64, 3, 3, 1, 1, 64, 3, 3, 1, 1, 64, 3, 3, 1, 1]

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 11:56:44.475340  2180 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 11:56:44.520220  2180 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [8]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [9]:
# def get_test_data(args):
#     '''
#     Input: (sess,offset)
#     '''
#     sess,i=args
#     test_data, test_ground_truth = audio_processor.get_data(
#         batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
#     return test_data,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      data_input,
      model_settings,
      model_architecture,
      model_size_info=model_size_info,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [11]:
tf.app.run(main=main)

W0119 11:56:45.279390  2180 deprecation_wrapper.py:119] From ..\libs\models.py:611: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0119 11:56:49.299734  2180 deprecation.py:323] From ..\libs\trainer.py:57: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0119 11:56:49.338629  2180 deprecation_wrapper.py:119] From ..\libs\trainer.py:58: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0119 11:56:49.342620  2180 deprecation_wrapper.py:119] From ..\libs\trainer.py:63: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0119 11:56:50.167219  2180 deprecation_wrapper.py:119] From ..\libs\trai

I0119 12:10:20.634217  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-53"
I0119 12:10:35.714516  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-54"
I0119 12:10:49.646923  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-55"
I0119 12:11:04.990203  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-56"
I0119 12:11:19.936026  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-57"
I0119 12:11:34.609105  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-58"
I0119 12:11:48.469433  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-59"
I0119 12:12:03.356777  2180 trainer.py:126] Step #60: rate 0.000500, accuracy 9.8%, cross entropy 3.398380
I0119 12:12:03.359769  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-60"
I0119 12:12:18.256188  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-61"
I0119 12:12:31.699553  2180 trainer.py:172] Saving to ".

I0119 12:29:15.654461  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-132"
I0119 12:29:30.995203  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-133"
I0119 12:29:44.563804  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-134"
I0119 12:29:56.710846  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-135"
I0119 12:30:07.707657  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-136"
I0119 12:30:19.179793  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-137"
I0119 12:30:30.489934  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-138"
I0119 12:30:41.463815  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-139"
I0119 12:30:53.237282  2180 trainer.py:126] Step #140: rate 0.000500, accuracy 14.5%, cross entropy 3.172089
I0119 12:30:53.240273  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-140"
I0119 12:31:03.454313  2180 trainer.py:172] S

I0119 12:47:00.586733  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-210"
I0119 12:47:12.800552  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-211"
I0119 12:47:28.566424  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-212"
I0119 12:47:42.309335  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-213"
I0119 12:47:56.920758  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-214"
I0119 12:48:10.422998  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-215"
I0119 12:48:25.481916  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-216"
I0119 12:48:40.132308  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-217"
I0119 12:48:54.923758  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-218"
I0119 12:49:09.942505  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-219"
I0119 12:49:23.227720  2180 trainer.py:126] Step #220: rate 

I0119 13:06:07.675080  2180 trainer.py:126] Step #290: rate 0.000500, accuracy 34.0%, cross entropy 2.655906
I0119 13:06:07.677075  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-290"
I0119 13:06:21.074288  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-291"
I0119 13:06:35.738585  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-292"
I0119 13:06:49.535079  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-293"
I0119 13:07:04.933227  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-294"
I0119 13:07:18.559131  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-295"
I0119 13:07:33.912641  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-296"
I0119 13:07:46.909171  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-297"
I0119 13:08:02.394113  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-298"
I0119 13:08:16.011971  2180 trainer.py:172] S

...

I0119 14:14:53.800837  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-602"
I0119 14:15:00.830386  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-603"
I0119 14:15:09.633639  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-604"
I0119 14:15:17.357783  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-605"
I0119 14:15:24.958593  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-606"
I0119 14:15:33.446794  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-607"
I0119 14:15:40.327600  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-608"
I0119 14:15:48.824301  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-609"
I0119 14:15:56.818539  2180 trainer.py:126] Step #610: rate 0.000500, accuracy 55.9%, cross entropy 1.892572
I0119 14:15:56.818539  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-610"
I0119 14:16:04.256693  2180 trainer.py:172] S

I0119 14:25:03.668920  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-680"
I0119 14:25:11.655147  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-681"
I0119 14:25:18.476111  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-682"
I0119 14:25:27.255777  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-683"
I0119 14:25:34.679076  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-684"
I0119 14:25:42.370177  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-685"
I0119 14:25:50.781856  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-686"
I0119 14:25:57.702764  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-687"
I0119 14:26:06.094813  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-688"
I0119 14:26:14.033313  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-689"
I0119 14:26:21.460055  2180 trainer.py:126] Step #690: rate 

I0119 14:35:22.847738  2180 trainer.py:126] Step #760: rate 0.000500, accuracy 62.9%, cross entropy 1.644969
I0119 14:35:22.847738  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-760"
I0119 14:35:30.826774  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-761"
I0119 14:35:38.845348  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-762"
I0119 14:35:45.564187  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-763"
I0119 14:35:54.094947  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-764"
I0119 14:36:01.422000  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-765"
I0119 14:36:08.832934  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-766"
I0119 14:36:17.050563  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-767"
I0119 14:36:24.054927  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-768"
I0119 14:36:32.354187  2180 trainer.py:172] S

I0119 14:45:24.468605  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-838"
I0119 14:45:31.024531  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-839"
I0119 14:45:39.614417  2180 trainer.py:126] Step #840: rate 0.000500, accuracy 63.7%, cross entropy 1.580715
I0119 14:45:39.614417  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-840"
I0119 14:45:47.455261  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-841"
I0119 14:45:54.497017  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-842"
I0119 14:46:03.035290  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-843"
I0119 14:46:10.004674  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-844"
I0119 14:46:17.786562  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-845"
I0119 14:46:26.188276  2180 trainer.py:172] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-846"
I0119 14:46:33.038855  2180 trainer.py:172] S

KeyboardInterrupt: 

In [ ]:
# save_checkpoint='../logs&checkpoint/ds_cnn/ckpt-30000'

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,model_size_info=model_size_info)

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-small-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)